# **1 Pada Gambar**
    - baca image -> flatten -> csv1
        - untuk baca path dari folder gambar bisa pakai pakege glob
    - kalau pada gambar data di baca dulu dalam pixel, trus di flatten, trus dimasukkan ke csv
    - kalau gambar bewarna itu ada 3 warna atau di sebut 3 chanel (RGB)

# **2 Pada Text**

In [1]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /home/rizki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/rizki/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from jcopml.pipeline import num_pipe, cat_pipe
from jcopml.utils import save_model, load_model

In [3]:
#Librari pemecah kata, ada satu lagi berdasarkan kata yaitu sent_tokenize
from nltk.tokenize import word_tokenize

#livrary untuk menemukan kata yang umum
from nltk.corpus import stopwords

#librari untuk memisahkan tanda baca
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

In [4]:
stopwords.words("indonesian")[:5]

['ada', 'adalah', 'adanya', 'adapun', 'agak']

In [5]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

## Import Data

In [6]:
df = pd.read_csv("../data/spam.csv")
df.head()

,Teks,label
0,[PROMO] Beli paket Flash mulai 1GB di MY TELKO...,1
1,2.5 GB/30 hari hanya Rp 35 Ribu Spesial buat A...,1
2,"2016-07-08 11:47:11.Plg Yth, sisa kuota Flash ...",1
3,"2016-08-07 11:29:47.Plg Yth, sisa kuota Flash ...",1
4,4.5GB/30 hari hanya Rp 55 Ribu Spesial buat an...,1


## Split Data

In [7]:
X = df["Teks"]
y = df["label"]

#karna klasifikasi kita pakai stratify
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914,), (229,), (914,), (229,))

## Training

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

#klw pakai Countvectorizer dia sistem nya (BoW bag of Word)
from sklearn.feature_extraction.text import TfidfVectorizer

from jcopml.tuning import random_search_params as rsp

In [9]:
pipeline = Pipeline([
    ("prep", TfidfVectorizer(tokenizer= word_tokenize, stop_words=sw_indo)),
    ("algo", LogisticRegression(solver='lbfgs', n_jobs=-1, random_state=42))
])

#Tuning
model = RandomizedSearchCV(pipeline, rsp.logreg_params, n_iter=50, cv=3, n_jobs=-1)
model.fit(X_train, y_train)

print(model.best_params_)
print(model.score(X_train, y_train),  model.score(X_test, y_test), model.best_score_)

/home/rizki/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/home/rizki/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/home/rizki/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/home/rizki/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words ma

{'algo__C': 13.961497016794214, 'algo__fit_intercept': True}
1.0 0.9781659388646288 0.96388769053782


In [10]:
y_pred = ["Ajukan segera pinjaman online dengan mudah tanpa potongan. Proses cepat cair. Dana 5 juta/500 juta. Info WA langsung nomor +6282123123123"]

In [11]:
print (model.predict_proba(y_pred))
print (model.predict(y_pred))

[[0.01818085 0.98181915]]
[1]


In [12]:
y_pred2 = ["Rizki Pinjam duit mu dong aku lagi butuh ni untuk biaya obat dari wahyu"]
model.predict(y_pred2)

array([0])

# **Predict Smilarity**

In [52]:
import pandas as pd
import numpy as np

#Librari pemecah kata, ada satu lagi berdasarkan kata yaitu sent_tokenize
from nltk.tokenize import word_tokenize

#livrary untuk menemukan kata yang umum
from nltk.corpus import stopwords

#librari untuk memisahkan tanda baca
from string import punctuation

sw_indo = stopwords.words("indonesian") + list(punctuation)

In [53]:
#klw pakai Countvectorizer dia sistem nya (BoW bag of Word)
from sklearn.feature_extraction.text import TfidfVectorizer

#Library untuk cek smilarity
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
df = pd.read_csv("../data/kompas.csv")
df.head()

,teks
0,Ginandjar Tetap Ditahan. Jaksa Agung Dilaporka...
1,Jakarta Dikangkangi Para Preman\nKALAU tak pun...
2,Penyimpangan di Setpres Seolah Terjadi Sekaran...
3,"Dibayarkan, Rapel Kenaikan Gaji Pegawai Pos\nK..."
4,"Stop Kekerasan, Elite agar Duduk Bersama\nSeju..."


## **Predict Smilarity**

In [93]:
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize, stop_words=sw_indo)
tfidf_matriks = tfidf.fit_transform(df["teks"])

/home/rizki/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [56]:
tfidf_matriks

<2008x548426 sparse matrix of type '<class 'numpy.float64'>'
	with 1173722 stored elements in Compressed Sparse Row format>

In [57]:
sim = cosine_similarity(tfidf_matriks[0], tfidf_matriks)
sim

array([[1.        , 0.00858328, 0.01060043, ..., 0.00856287, 0.00677808,
        0.01513341]])

In [19]:
# Urutkan value dari terkecil ke terbesar tetapi yang di return index array nya bukai nilai array nya
sim.argsort()

array([[ 932, 1131, 1593, ...,  215,  144,    0]])

In [58]:
df["teks"][144][:200]

'Kejaksaan Agung Terbitkan Surat Penahanan Baru\nKejaksaan Agung (Kejagung) akhirnya menerbitkan surat perintah penahanan yang baru terhadap mantan Menteri Pertambangan dan Energi Ginandjar Kartasasmita'

In [59]:
df["teks"][215][:200]

'Kuasa Hukum Ginandjar Bertahan di Rutan\nSejumlah kuasa hukum Ginandjar Kartasasmita hingga hari Selasa (17/4) pukul 22.00 masih bertahan di ruang tahanan (rutan) Kejaksaan Agung (Kejagung). Selasa pag'

# **Ekstrak Keyword**

In [65]:
vocab = tfidf.get_feature_names()
vocab[-10:]

['zuniga memilih',
 'zunnatul',
 'zunnatul mafruhah',
 'zurich',
 'zurich northholt',
 'zw',
 'zw suparman',
 'zw tim',
 'zx',
 'zx diserbu']

In [66]:
ektrak = tfidf_matriks.toarray()[0].argsort()
ektrak

array([274212, 365469, 365468, ..., 386379, 436652, 169219])

In [67]:
vocab[169219]

'ginandjar'

In [68]:
[vocab[idx] for idx in reversed(ektrak[-10:])]

['ginandjar',
 'putusan',
 'penahanan',
 'hukum ginandjar',
 'kuasa hukum',
 'rusman',
 'kejaksaan',
 'hakim rusman',
 'kuasa',
 '9 april']

## **Cek Highlit Berita**

In [91]:
berita1 = """
Menteri Komunikasi dan Informatika Johnny G. Plate mengatakan pemerintah telah menyiapkan laman vaksinIn.dto.kemkes.go.id untuk para WNI maupun WNA untuk mendaftar dan mengajukan verifikasi.
Proses verifikasi akan dikonfirmasi melalui email yang sudah didaftarkan di laman tersebut dalam waktu maksimal 3 hari kerja.
"Setelah itu pendaftar harus masuk ke dalam aplikasi PeduliLindungi untuk mengklaim sertifikat vaksin yang nanti muncul setelah diverifikasi," ujar Johnny, Kamis (16/9).
Setelah itu, lanjutnya, WNI atau WNA tersebut bisa menggunakan aplikasi PeduliLindungi untuk mengakses berbagai tempat fasilitas umum.
Berkas yang harus disiapkan bagi WNI berupa KTP dengan NIK, ID yang dipakai untuk verifikasi adalah NIK dan Kartu Vaksinasi. Verifikasi akan dilakukan oleh Kementerian Kesehatan.
Adapun berkas yang harus disiapkan oleh WNA, lanjut Johnny, adalah izin diplomatik dari Kementerian Luar Negeri atau izin tinggal dari imigrasi dan kartu vaksinasi. ID yang dipakai untuk verifikasi adalah nomor paspor.
Verifikasi bagi WNA dengan izin diplomatik dilakukan oleh Kementerian Luar Negeri, sedangkan verifikasi bagi WNA dengan izin tinggal masih dalam proses finalisasi antara Kemenkes dengan Kemenlu.
Selain itu, WNI dan WNA yang mendapatkan vaksinasi di luar negeri juga dapat menggunakan aplikasi PeduliLindungi untuk memudahkan dalam memperlancar akomodasi ke fasilitas publik.
"Penambahan fitur ini dapat membantu pemerintah dalam melakukan tracing dengan cepat untuk mengendalikan pandemi," ujarnya.
"""

In [94]:
tfidf_matriks1 = tfidf.transform([berita1])
tfidf_matriks1

<1x548426 sparse matrix of type '<class 'numpy.float64'>'
	with 69 stored elements in Compressed Sparse Row format>

In [101]:
ektrak1 = tfidf_matriks1.toarray()[0].argsort()

In [100]:
[vocab[idx] for idx in reversed(ektrak1[-10:])]

['wna',
 'verifikasi',
 'wni',
 'johnny',
 'aplikasi',
 'vaksinasi',
 'id',
 'izin tinggal',
 'izin',
 'kementerian']

In [102]:
# atau dapat dibuat fungsi sebagai berikut

def HighlitBerita (berita, tfidf, jlhKataHighlit = 10):
    tfidf_matriks1 = tfidf.transform([berita])
    ektrak1 = tfidf_matriks1.toarray()[0].argsort()
    return [vocab[idx] for idx in reversed(ektrak1[-jlhKataHighlit:])]

In [103]:
HighlitBerita(berita1)

['wna',
 'verifikasi',
 'wni',
 'johnny',
 'aplikasi',
 'vaksinasi',
 'id',
 'izin tinggal',
 'izin',
 'kementerian']